# Running Auto Correlation with Talon Deployer and BITE
###### Last Updated: 01/04/24

This demo will show the basic operation of auto correlation using the new Deployer and BITE TANGO devices currently deployed in this repository. With this notebook, all TANGO commands and attribute changes are made via a [TANGO DeviceProxy](https://pytango.readthedocs.io/en/stable/client_api/device_proxy.html) but the overall steps should be the same for using the JIVE interface or Taranta web interface.

## Pre-requisites


First, this notebook assumes you have a running environment launched from a pipeline, in particular it assumes you are running off one launched from the [SKA-mid-psi](https://gitlab.com/ska-telescope/ska-mid-psi) pipeline. Secondly, for ease of dev work, it also assumes you are using a virtual env. This notebook was made with Python 3.10 in mind.

Finally, make sure all requirements are installed via [poetry]().

### Running on PSI


To Run on PSI, once running the launch step from the PSI pipeline, grab your booted namespace's name. You can run a check on the ns as well to make sure the pods are ready.

In [ ]:
!kubectl get ns | grep ska-mid-psi

To make future work a bit easier we can load this into a NS var.

In [ ]:
# Change this var to be the non-spd namespace with your username in it
NS = "ci-ska-mid-psi-1228919416-alexschell"

url = "https://142.73.34.170/"+NS+"/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run"
print("You can monitor board status using: https://142.73.34.170/{}/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run".format(NS))

We can then check to ensure the pods have spun up successfully.

In [ ]:
!kubectl -n $NS get pods

## Initial Setup/Using Deployer Device

First we want to set up the target board that we will be using for the auto correlation. While this can be multiple boards, for now we will only need one. Punch out the board(s) you have access to and set the `TARGET_BOARD` to assign it to be used in future steps.

The `TANGO_HOST` will be created based of the namespace you set earlier.

In [ ]:
import os
# Set to the boards you wish to configure.
TARGET_BOARD = [1]

TANGO_HOST = "databaseds-tango-base."+NS+".svc.cluster.local:10000"
print("Will be using HOST: ", TANGO_HOST)

os.environ["TANGO_HOST"] = TANGO_HOST

We then load in the locations of local files for configuration. If you wish to use custom files you can change the folders pointed to, otherwise this code block can be run without edits.

In [ ]:
# Get files from configs folder in repo
CURRDIR = os.path.dirname(os.getcwd())
CORRELATOR_FILE_LOCATION = os.path.abspath(os.path.join(CURRDIR,"configs/auto-correlation"))
COMMON_FILE_LOCATION = os.path.abspath(os.path.join(CURRDIR,"configs/common"))

Next, we pass in the SLIM mesh config files by copying them to the namespace, these files should be in the json_files storage folder. Custom files here are not required, but if needed, the following two code blocks can be used.

In [ ]:
# Change this value if you want to specify a certain slim directory for files
slim_fs_config = ""
slim_vis_config = ""
if slim_fs_config != "" and slim_vis_config != "" :
    fs_config_path = os.path.join(CORRELATOR_FILE_LOCATION, slim_fs_config)
    vis_config_path = os.path.join(CORRELATOR_FILE_LOCATION, slim_vis_config)    
else:
    print("SLIM will use defaults for this test.")

In [ ]:
!"kubectl cp {fs_config_path} {NS}/ds-cbfcontroller-controller-0:/app/mnt/slim/fs_slim_config.yaml"
!"kubectl cp {vis_config_path} {NS}/ds-cbfcontroller-controller-0:/app/mnt/slim/vis_slim_config.yaml"

For this demo, we will interact with the TANGO devices via a device proxy, which will allow us to pass commands into them as we would in the UI.

In [ ]:
from PyTango import DeviceProxy, Database

# Setup the device proxies targeting bite and deployer
db = Database()
deployer_tango = DeviceProxy("mid_csp_cbf/ec/deployer")

#Check the devices initially deployed to the database
devices = db.get_device_exported("*")
print(devices)
# Make sure the deployer device is set to ON
deployer_tango.On()
print(deployer_tango.state())

Next we load in the hardware configuration depending on the talon boards selected.

In [ ]:
if 4 in TARGET_BOARD or 5 in TARGET_BOARD:
    print("Using swap for higher number talons")
    config = "hw_config/hw_config_sawp_psi.yaml"
else:
    print("Using standard HW config")
    config = "hw_config/hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(COMMON_FILE_LOCATION, config)

In [ ]:
!kubectl cp $HW_CONFIG_FILE $NS/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

Currently using this step as a workaround for the deployer not generating a file that later steps use.

In [ ]:
!kubectl exec -ti -n $NS ec-deployer -- python3 midcbf_deployer.py --generate-talondx-config --boards=1
!kubectl exec -ti -n $NS ec-deployer -- python3 midcbf_deployer.py --download-artifacts
!kubectl exec -ti -n $NS ec-deployer -- python3 midcbf_deployer.py --config-db

First, we set the target talon boards we want to set up our configuration for. In Jive/Taranta, this would be configured by manually writing the attribute via the UI. Multiple boards can be targeted.

In [ ]:
deployer_tango.targetTalons = TARGET_BOARD
print(deployer_tango.targetTalons)

With this set, we can then run the configuration command by calling the generate_config_jsons command.

In [ ]:
deployer_tango.generate_config_jsons()

We then get the device artifacts from the [artifact repository](https://artefact.skao.int/#browse/browse:helm-internal) by running the command via TANGO. This step may take some time as it downloads multiple devices.

In [ ]:
deployer_tango.set_timeout_millis(200000)
try:
    deployer_tango.download_artifacts()
except Exception as e:
    print(e)
    print("Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes.")
deployer_tango.set_timeout_millis(4500)

To check that the artifacts downloaded successfully, we want to check that the following returns something like `INFO|Dummy-2|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading`. 

In [ ]:
!kubectl logs -n $NS ds-deployer-deployer-0 -f | grep 'Finished downloading'

Finally, we can configure the TANGO database with all the tango devices we just downloaded using the ConfigDB command.

In [ ]:
deployer_tango.configure_db()

The TANGO database should now be configured with all the devices needed for the next step, running the BITE device.

In [ ]:
print(*db.get_device_exported("*").value_string, sep="\n")

## Setting Controller settings

Now that we have the controller device deployed, we can use it to prep for further steps.

In [ ]:
CBF_tango = DeviceProxy("mid_csp_cbf/sub_elt/controller")
print(CBF_tango.status())
control_tango = DeviceProxy("mid-csp/control/0")
print(control_tango.status())
subarray_tango = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")
print(subarray_tango.status())
elt_controller_tango = DeviceProxy("mid_csp_cbf/sub_elt/controller")
print(elt_controller_tango.status())

We can then set adminMode to 0 (ONLINE), allowing us to run commands, and set simulationMode to 0 (FALSE).

In [ ]:
# Set admin mode to online and sim mode to off on the cbf contoller
CBF_tango.adminMode = 0 
CBF_tango.simulationMode = 0
# Set admin mode to online and sim mode to off for subarray
subarray_tango.adminMode = 0 
subarray_tango.simulationMode = 0
# Set relevant values on the mid-csp controller
control_tango.adminMode = 0 
control_tango.simulationMode = 0
control_tango.cbfSimulationMode = 0

elt_controller_tango.adminMode = 0
elt_controller_tango.simulationMode = 0

print(CBF_tango.State())
print(control_tango.State())

if CBF_tango.read_attribute("adminMode").value == 0 and CBF_tango.read_attribute("simulationmode").value == 0:
    print ("Set values successfully!")
elif CBF_tango.read_attribute("adminMode").value == 0 and CBF_tango.read_attribute("simulationmode").value == 1:
    print ("Set to simulation mode on.")
else:
    print ("Error, couldn't set values!")


Checking the status dashboard, it should now display that all devices are both OFF and that the simulationstate is FALSE.

Next, we load in a inital values parameters and pass it to the controller, to do this we read in JSON file and pass it as a DevString to the relevant device command:

In [ ]:

init_sys_param_file = os.path.join(COMMON_FILE_LOCATION, "sys_params/initial_system_param.json")

with open(init_sys_param_file) as init_file:
    data = init_file.read()
# Use the InitSysParam command and feed in data
    
# add in check for off state before writing
upload_result = CBF_tango.InitSysParam(data)
print(upload_result[1])
print(subarray_tango.sysParam)

We then load in the dish configurations to the mid csp Controller:

In [ ]:
# Read in Json file from set configs folder
dish_config_file = os.path.join(COMMON_FILE_LOCATION,"sys_params/load_dish_config.json")
with open(dish_config_file) as file: 
    dish_config = file.read()
control_tango.LoadDishCfg(dish_config)

Now, we turn ON the Controller by passing it the device we want to turn on, and letting it run for 45ms to give the boards time to power on.

In [ ]:
from time import sleep
print(control_tango.status())
#CBF_tango.Init()
#CBF_tango.On()
#control_tango.Init()
control_tango.set_timeout_millis(45000)
target = ["mid_csp_cbf/sub_elt/controller"]
on_return = control_tango.On(target)
print(on_return)
sleep(50)
print(control_tango.status())

After running this step, check with the Tarnata dashboard to check that the boards are started.

TODO: Add in HW swap for using higher # Talon boards.

## Running Commands Through the BITE Device

Now that the BITE tango Device has been deployed via the deployer, we can use it to configure tests. First we check the device is running.

In [ ]:
target_mac_address = "08:c0:eb:9d:47:78"
bite_tango = DeviceProxy("mid_csp_cbf/ec/bite")
# Running this should return RUNNING
print(bite_tango.State())

Now that we have confirmed it is running, we load in what board we want to configure.

In [ ]:
bite_tango.boards = TARGET_BOARD
bite_tango.bite_mac_address = target_mac_address

For now, we can use the defaults and simply call the write command for the test configs. This should return the configuration for each board passed in.

In [ ]:
#print(bite_tango.command_inout("generate_bite_data"))

# For now run the backup
standin_board = TARGET_BOARD[0]
print(standin_board)
!kubectl exec -ti -n $NS ec-bite -- python3 midcbf_bite.py --talon-bite-config --boards=$standin_board --bite_mac_address=$target_mac_address

In [ ]:
!kubectl -n $NS logs ds-bite-bite-0 | grep generate_bite_data

With the bite data generated, we can then start generating LSTV playback data:

In [ ]:
bite_tango.start_lstv_replay()

We can then check the logs to ensure the LSTV geneartion runs correctly.

In [ ]:
!kubectl -n $NS logs ds-bite-bite-0 -f

## Assigning Resources

Next, we use the relevant subarray device to assign resources. First, as with the other devices, we establish a DeviceProxy to connect to it. We also read in the assign_resources file to load in using the command. 

In [ ]:
subarray_tango = DeviceProxy("mid-csp/subarray/01")
print(subarray_tango.State())
assign_resources_file = os.path.join(CORRELATOR_FILE_LOCATION, "assign_resources.json")
with open(assign_resources_file) as init_file:
    resource_data = init_file.read()
subarray_tango.On()
print(resource_data)
print(subarray_tango.State())

Next,we run the actual command to pass in the resources:

In [ ]:
subarray_tango.AssignResources(resource_data)

## SDP Setup

Next, we run a few commands to set up the Science Data Processor (SDP), utilizing the sdp namespace

In [ ]:
sdp_namespace = NS+"-sdp"
print(sdp_namespace)
!kubectl -n $sdp_namespace get pods

Now, we assign resources as we did with the subarray:

## Cleanup

Now that we're done, free up dev resources on PSI by deleting your ns.

In [ ]:
!kubectl delete ns $NS
!kubectl delete ns $NS-sdp

In [ ]:
!kubectl get ns | grep ska-mid-psi